## If you have LLM and tool , how you will connect them.

## How LLM call a tool when it needs to call.

# 1. Tool Binding

* **Tool Binding** is the step where you **register tools** with a Language Model (LLM) so that

1. The LLM knows **what tools are available.**

2. It knows **what each tool does** (via description)

3. It knows **what input format to use** (via schema)

In [1]:
# !pip install langchain langchain-core langchain-community pydantic duckduckgo-search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 97.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 1.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.2.1
    Uninstalling langchain-core-1.2.1:
      Successfully uninstalled langchain-core-1.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-col

In [2]:
# !pip install -U ddgs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 5.8 MB/s eta 0:00:00


In [3]:
# !pip install langchain-experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.1/210.1 kB 5.3 MB/s eta 0:00:00


In [6]:
!pip install langchain-huggingface python-dotenv langchain-text-splitters

In [40]:
!pip install langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.7/84.7 kB 2.8 MB/s eta 0:00:00


In [ ]:
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"]= "...."

In [73]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint, HuggingFaceEmbeddings

from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

## 1. tool create

In [9]:
# tool create

@tool
def multiply(a:int, b:int)-> int:
  """this tool will return Multiplication between a and b """
  return a*b

In [10]:
multiply.invoke({'a':5, 'b':8})

40

In [13]:
print(multiply.name)
print(multiply.description)
print(multiply.args)
print(multiply.args_schema.model_json_schema())

multiply
this tool will return Multiplication between a and b
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}
{'description': 'this tool will return Multiplication between a and b ', 'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'multiply', 'type': 'object'}


## 2. tool binding

* now these llm has a fn called **bind_tools** and through that you can bind any number of tools (list of those tools has to provide)

## Note
* all LLM do not have that power to bind any tool; there are some LLM which has this capability to bind tools (below are there lists)

In [35]:
llm = HuggingFaceEndpoint(
    repo_id="tiiuae/falcon-7b-instruct",
    task="text-generation",
    provider="hf-inference",
    huggingfacehub_api_token= os.getenv("HUGGINGFACEHUB_API_TOKEN")
)

model= ChatHuggingFace(llm= llm)


In [36]:
llm_with_tools= model.bind_tools([multiply]) # now this is also a llm but it has a tool

# now in future if model feels that it has to do multiplication so it can call that tools (now that power is in hand of llm)

# Tool calling

* **Tool Calling** is the process where **LLM (language model)** decide, during a conversation or task, that it needs to **use a specific tool (function)** -- and generates a structured output with:

* * the **name of the tool**
* * and the arguments to call it with

* The LLM does not actually run the tool -- it just suggests the tool and the input arguments. The **actual execution is handled by LangChain or you.**

* eg "What's 8 multiplication by 7?" \

the llm responds with a **tool call**

* {
  "tool": "multiply",
  "args": {"a":8, "b":7}
}

* so this will be given by llm during tool calling main tool will be happend by langchain using this args

In [ ]:
# like see this hugging face model doesn't use tool , we have to use any other model

In [ ]:
os.environ["OPENAI_API_KEY"]= "..."

In [41]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI()

In [43]:
llm_with_tools= llm.bind_tools([multiply])

In [44]:
llm_with_tools.invoke("can you multiply 3 with 10")

AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 60, 'total_tokens': 92, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Cs4NFXeZcIFinefgKrBtB8lmtRv0P', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b698f-555e-71f3-806f-cb71d347293f-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 10}, 'id': 'call_Ppi4MhWsLMwj9BgHIKuOX32S', 'type': 'tool_call'}], usage_metadata={'input_tokens': 60, 'output_tokens': 32, 'total_tokens': 92, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## you can see the content thing is empty but the you can see **tool_calls**

In [45]:
llm_with_tools.invoke("can you multiply 3 with 10").tool_calls # you can see this is all list of tools which it used

[{'name': 'multiply',
  'args': {'a': 3, 'b': 10},
  'id': 'call_kMRS4iLdw1ImyF0BxJe6VKuD',
  'type': 'tool_call'}]

In [47]:
# so llm thought that for solving that particular question it has to call that tool and also ready all args for calling that tool

# since llm can run that tool it can ony **suggest** and that running part has to be ron by langchain itself
# that why content was empty

# and this also good bz you can't totally trust on llm bz it can call anything ;; you just asking advice from llm with **input**
## that is call tool calling (this suggestion by llm with **ready input**)

## now time to execute tool


* we have leant tool builing , tool binding, tool calling
## Tool execution:
* **Tool execution** is the step where the **actual Python function (tool)** is run using the input arguments tht the **LLM suggested during tool calling**

* The LLM says:
* * "Hey, call the multiply tool with a=8 and b=7."

* * **Tool execution** is when you or LangChain actually run: multiply(a=8, b=7) -> and get the result : 56


In [48]:
result= llm_with_tools.invoke("can you multiply 3 with 10")

In [49]:
result

AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 60, 'total_tokens': 77, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Cs4iRngjZcFtMU2u9jfNTOkmjEoNF', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b69a3-6331-7da0-815a-6166abb09ff6-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 10}, 'id': 'call_9Hc2Z5xw8OEcKY1OZLAGKAjj', 'type': 'tool_call'}], usage_metadata={'input_tokens': 60, 'output_tokens': 17, 'total_tokens': 77, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [51]:
result.tool_calls[0]

{'name': 'multiply',
 'args': {'a': 3, 'b': 10},
 'id': 'call_9Hc2Z5xw8OEcKY1OZLAGKAjj',
 'type': 'tool_call'}

In [59]:
## now tool execution process by me manually getting input from llm

multiply.invoke(result.tool_calls[0]['args'])

30

In [61]:
## if i send whole tool_calls then
# we are also getting ans in content but diff is all the things are packed around it
# which is called "ToolMessage" (lly like AiMessage, SystemMessage, HumanMessage)

multiply.invoke(result.tool_calls[0])

# banifit is that you can now send this toolMessage to llm directly;; and it will know that this is tool ans


ToolMessage(content='30', name='multiply', tool_call_id='call_9Hc2Z5xw8OEcKY1OZLAGKAjj')

## Now try to do same things with good manner

In [63]:
query= HumanMessage("can you multiply 3 with 10")

messages= [query]

In [64]:
result= llm_with_tools.invoke(messages)
result

AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 60, 'total_tokens': 77, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Cs4wj3fGjjEZXzc8SmNibYASAH390', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b69b0-ebae-7e70-875b-3d570b3c4842-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 10}, 'id': 'call_3Vzkas3OuaM5uN9XsnzZAHZX', 'type': 'tool_call'}], usage_metadata={'input_tokens': 60, 'output_tokens': 17, 'total_tokens': 77, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [65]:
messages.append(result)
messages

[HumanMessage(content='can you multiply 3 with 10', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 60, 'total_tokens': 77, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Cs4wj3fGjjEZXzc8SmNibYASAH390', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b69b0-ebae-7e70-875b-3d570b3c4842-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 10}, 'id': 'call_3Vzkas3OuaM5uN9XsnzZAHZX', 'type': 'tool_call'}], usage_metadata={'input_tokens': 60, 'output_tokens': 17, 'total_tokens': 77, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token

In [69]:
tool_result= multiply.invoke(result.tool_calls[0])
messages.append(tool_result)
messages

[HumanMessage(content='can you multiply 3 with 10', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 60, 'total_tokens': 77, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Cs4wj3fGjjEZXzc8SmNibYASAH390', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b69b0-ebae-7e70-875b-3d570b3c4842-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 10}, 'id': 'call_3Vzkas3OuaM5uN9XsnzZAHZX', 'type': 'tool_call'}], usage_metadata={'input_tokens': 60, 'output_tokens': 17, 'total_tokens': 77, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token

In [71]:
final_result= llm_with_tools.invoke(messages)
final_result

AIMessage(content='The result of multiplying 3 by 10 is 30.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 85, 'total_tokens': 99, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Cs51NTyMWmnxWcfSD7bN6SghOIoAx', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b69b5-4e08-7460-9085-4085edac2de4-0', usage_metadata={'input_tokens': 85, 'output_tokens': 14, 'total_tokens': 99, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## Currency Conversion Tool

* now try to build a application , where we give a power to our LLM that it can do **real time currency conversion**

* so for that we use a python fn inside it we will use api (**exchangeRate-API**) where you have to give source currency and target currency so it will give you that conversion factor at current time

In [ ]:
# tool create
from langchain_core.tools import InjectedToolArg
from typing import Annotated

@tool
def get_conversion_factor(base_currency:str, target_currency:str) -> float:
  """
  This function fetches the currency conversion factor between a given base currency and target currency
  """
  url= f"https://v6.exchangerate-api.com/v6/.../pair/{base_currency}/{target_currency}"

  # to sent http request to this url , we will use **requests**
  response= requests.get(url)
  return response.json()

print(get_conversion_factor.invoke({"base_currency":"USD","target_currency":"INR"}))


{'result': 'success', 'documentation': 'https://www.exchangerate-api.com/docs', 'terms_of_use': 'https://www.exchangerate-api.com/terms', 'time_last_update_unix': 1766966401, 'time_last_update_utc': 'Mon, 29 Dec 2025 00:00:01 +0000', 'time_next_update_unix': 1767052801, 'time_next_update_utc': 'Tue, 30 Dec 2025 00:00:01 +0000', 'base_code': 'USD', 'target_code': 'INR', 'conversion_rate': 90.0348}


In [80]:
## another tool for multiplication

@tool
def convert(base_currency_value:float, conversion_rate: Annotated[float, InjectedToolArg])-> float:
  """ will find final calculated total target currency for that base currency amount and conversion rate"""

  return base_currency_value * conversion_rate

In [84]:
convert.invoke({"base_currency_value":12.0, "conversion_rate":90.0})

1080.0

In [85]:
# tool binding
llm= ChatOpenAI()

llm_with_tools_for_currency_conver= llm.bind_tools([get_conversion_factor, convert])

In [87]:
query= HumanMessage("Convert 13 USA currency in INR currency.")

messages= [query]

results= llm_with_tools_for_currency_conver.invoke(messages)

In [94]:
messages.append(results)
results

AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 109, 'total_tokens': 131, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Cs5vzir0y7APutRs6tzYYkSf4eGnf', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b69ea-ddd0-7693-bb7b-eee67ac06c32-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'base_currency': 'USD', 'target_currency': 'INR'}, 'id': 'call_pyvn8GEeSdEGVQP73IoIU0oX', 'type': 'tool_call'}], usage_metadata={'input_tokens': 109, 'output_tokens': 22, 'total_tokens': 131, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [91]:
results.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'USD', 'target_currency': 'INR'},
  'id': 'call_pyvn8GEeSdEGVQP73IoIU0oX',
  'type': 'tool_call'}]

In [95]:
conversion_rate= get_conversion_factor.invoke(results.tool_calls[0])
messages.append(conversion_rate)
conversion_rate

ToolMessage(content='{"result": "success", "documentation": "https://www.exchangerate-api.com/docs", "terms_of_use": "https://www.exchangerate-api.com/terms", "time_last_update_unix": 1766966401, "time_last_update_utc": "Mon, 29 Dec 2025 00:00:01 +0000", "time_next_update_unix": 1767052801, "time_next_update_utc": "Tue, 30 Dec 2025 00:00:01 +0000", "base_code": "USD", "target_code": "INR", "conversion_rate": 90.0348}', name='get_conversion_factor', tool_call_id='call_pyvn8GEeSdEGVQP73IoIU0oX')

In [96]:
results_after_getting_rate= llm_with_tools_for_currency_conver.invoke(messages)
results_after_getting_rate

AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 275, 'total_tokens': 298, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Cs61dDcMdXxCjx4vvrGKEwEjhfjrf', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b69f0-342b-7763-982e-0d8c47f7ed38-0', tool_calls=[{'name': 'convert', 'args': {'base_currency_value': 13, 'conversion_rate': 90.0348}, 'id': 'call_k6znLB8bx9RgWGxn2yWJalDE', 'type': 'tool_call'}], usage_metadata={'input_tokens': 275, 'output_tokens': 23, 'total_tokens': 298, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [97]:
messages.append(results_after_getting_rate)

In [98]:
results_after_getting_rate.tool_calls

[{'name': 'convert',
  'args': {'base_currency_value': 13, 'conversion_rate': 90.0348},
  'id': 'call_k6znLB8bx9RgWGxn2yWJalDE',
  'type': 'tool_call'}]

In [99]:
final_converted_currency= convert.invoke(results_after_getting_rate.tool_calls[0])
final_converted_currency

ToolMessage(content='1170.4524000000001', name='convert', tool_call_id='call_k6znLB8bx9RgWGxn2yWJalDE')

In [100]:
messages.append(final_converted_currency)

In [101]:
final_llm_outcome_reg_final_amount= llm_with_tools_for_currency_conver.invoke(messages)
final_llm_outcome_reg_final_amount

AIMessage(content='13 USD is equivalent to 1170.45 INR.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 313, 'total_tokens': 327, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Cs65PnvzQZtBOXnuhFkaYYmJJbAQj', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b69f3-c5c0-71f3-80a4-63682fa0f8d9-0', usage_metadata={'input_tokens': 313, 'output_tokens': 14, 'total_tokens': 327, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [102]:
messages.append(final_llm_outcome_reg_final_amount)

In [103]:
final_llm_outcome_reg_final_amount.content

'13 USD is equivalent to 1170.45 INR.'

## Injected tool argument ;; so llm will understand the tools dependencies

* through that we can tell llm that any tool arg is **injected tool args** it's depend other tool outcome

## through that we are telling llm --> "LLM, do not try to fill this argument." (user telling) . "I (the developer/runtime) will inject this value after running earlier tools."

In [ ]:
import json

for tool_call in results.tool_calls:
  # execute the 1st tool and get the value of conversion rate
  if tool_call['name']== "get_conversion_factor":
    get_conversion_factor= get_conversion_factor.invoke(tool_call) ## it will give you json obj so have to convert in dic;; pro was you can't access json key directly
    conversion_rate= json.load(get_conversion_factor)['conversion_rate']
    messages.append(get_conversion_factor)


  # execute the 2nd tool using the conversion rate from tool 1
  if tool_call['name']== 'convert':
    # since we didn't have sec arg but had first one , and only need to call bz we also have gotten sec arg
    # so try to get its args
    args= tool_call['args']
    args['conversion_rate']= conversion_rate ## see after ward we are injecting new arg that why it is called injected_tool_arg
    conversion_amount = convert.invoke(args)
    messages.append(conversion_amount)
    print(conversion_amount)



# is this Ai agent -> No

* Ai agent characteristic -> that would be autonomus means ;; it will break down the works in steps and try to do itself

* it doesn't have to take help in any steps

* if a agent do same then the working flow would be

1. User says: 'convert 10 USD to INR'

2. LLM thinks: 'I don't know the rate. First, let me call get_conversion_factor.'

3. Tool result comes: 90.0

4. LLM looks at result THINK again: "now i know the rate, next i should call convert with 10 and 90.0

5. Tool result comes ...

6. LLM summarizes: '10 USD is .. INR at current rate."